In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import os, glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
from skimage.measure import block_reduce
from numpy.lib.stride_tricks import as_strided
plt.style.use([hep.style.ROOT, hep.style.firamath])
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LogNorm, ListedColormap, LinearSegmentedColormap
import matplotlib.patches as mpatches
# Define the CMS color scheme
cms_colors = [
    (0.00, '#FFFFFF'),  # White
    (0.33, '#005EB8'),  # Blue
    (0.66, '#FFDD00'),  # Yellow
    (1.00, '#FF0000')   # red
]

# Create the CMS colormap
cms_cmap = LinearSegmentedColormap.from_list('CMS', cms_colors)
from numpy.lib.stride_tricks import as_strided

plt.style.use(hep.style.CMS)  # Apply CMS plot style


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-2a_n85a2 because the default path (/uscms/home/bbbam/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# out_dir='data/plot_H_AA_4Tau_M_10_signal_kinematics'
out_dir='../analysis_run3/AN_Note_Plot/Images_pixels_intensity'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
save = True

In [18]:

def plot_cms_hist_per_channel(
    file,
    dataset_name="all_jet",
    sample_size=50000,
    bins=100,
    log_scale=True,
):
    """
    Plot CMS-styled histograms of pixel intensities for each channel.
    
    """
    layers =['Tracks_pt', 'Trac_dZSig', 'Trac_d0Sig', 'ECAL_energy', \
         'HBHE_energy', 'PixAtEcal_1','PixAtEcal_2', 'PixAtEcal_3', 'PixAtEcal_4', 'TibAtEcal_1',\
         'TibAtEcal_2', 'TobAtEcal_1','TobAtEcal_2']
    x_labels = ['Track Pt [Gev]', 'dZ [mm]', 'd0 [mm]', 'Ecal Energy [GeV]', 'Hcal energy [GeV]', 'Pixel layer 1 hits', 'Pixel layer 2 hits', 'Pixel layer 3 hits',\
                'Pixel layer 3 hits', 'TIB layer 1 hits', 'TIB layer 2 hits', 'TOB layer 1 hits', 'TOB layer 2 hits']
    zero_supression_min = np.array([0.001, 0.0001, 0.0001, 0.001, 0.001])
    zero_supression_max = np.array([1000, 20, 10, 500, 100])
        
    with h5py.File(file, "r") as f:
        data = f[dataset_name]
        n_events, n_channels, h, w = data.shape
        print(f"Loaded dataset '{dataset_name}' with shape: {data.shape}")

        # Random sampling
        if n_events > sample_size:
            idx = np.random.choice(n_events, sample_size, replace=False)
            idx.sort()  # Required for h5py
            sample = data[idx, :, :, :]
        else:
            sample = data[:]

        sample = np.array(sample)

    # --- Plot individual histograms per channel ---
    for i in range(n_channels):
        fig, ax = plt.subplots(dpi=300)
        pixels = sample[:, i, :, :].ravel()
        if i==0:
            pixels[pixels > zero_supression_max[i]] = 0
            
        pixels = pixels[pixels != 0]
        if (i>4 and i<8):
            bins=np.arange(0,16,1)
        if i>7:
            bins=np.arange(0,26,1)
        ax.hist(pixels, bins=bins, histtype='step',color="steelblue")
        ax.set_xlabel(f"{x_labels[i]}")
        ax.set_ylabel("Pixel Counts")

        if log_scale:
            ax.set_yscale("log")

        # CMS label
        hep.cms.label(
            llabel="Simulation",
            rlabel=f"{layers[i]}[{i+1}]              13.6 TeV",
            loc=0,
            ax=ax,
        )

        plt.tight_layout()
        plt.savefig(f'{out_dir}/mass_regression_sample_image_pixels_intensity_{layers[i]}.pdf',  bbox_inches='tight', dpi=300, facecolor = "w")
        plt.show()
        plt.close(fig)  # free memory
        


if __name__ == "__main__":
    file = "../analysis_run3/Data_for_plots/IMG_aToTauTau_Hadronic_m3p6To18_pt30T0300_unbiased_combined_valid.h5"
    plot_cms_hist_per_channel(file)


Loaded dataset 'all_jet' with shape: (388745, 13, 125, 125)


MemoryError: Unable to allocate 37.8 GiB for an array with shape (50000, 13, 125, 125) and data type float32

### Layesrs in images